# 3.1.1 PARAMETER IMPACT ANALYSIS

## Phân tích ảnh hưởng của tham số lên hiệu suất thuật toán

Notebook này phân tích ảnh hưởng của các tham số chính đến chất lượng giải pháp và hiệu suất:
- **GBFS (Greedy Best-First Search)**: Max States - giới hạn số trạng thái khám phá trong quá trình tìm kiếm
- **BPSO**: Swarm Size (kích thước bầy đàn), Iterations (số vòng lặp)

**✅ GBFS IMPLEMENTATION:**
- Code hiện tại implement **ĐÚNG Greedy Best-First Search** theo lý thuyết
- Có **state tree** representation với KnapsackState class
- Sử dụng **priority queue (heapq)** cho open set
- Có **closed set** để tránh revisit states
- **State expansion**: thử add từng item để tạo states mới
- Sử dụng **cùng fitness function với BPSO**: `fitness = alpha * f1_norm + beta * f2_norm - penalty`
  - f1 = Total Revenue (normalized), alpha = 0.7
  - f2 = Region Coverage (normalized), beta = 0.3
  - penalty = 10.0 * overflow_ratio nếu vượt capacity

**Key Features:**
- Max States parameter **ĐƯỢC SỬ DỤNG** để giới hạn search space
- GBFS explores state tree theo hướng fitness cao nhất
- Cho phép 20% overflow trong exploration (sau đó penalty)
- True search algorithm với exploration và exploitation

In [ ]:
# Import các thư viện cần thiết
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Add project root to path
sys.path.insert(0, '../../')
from src.visualization import AdvancedKnapsackVisualizer

# Create visualizer instance
visualizer = AdvancedKnapsackVisualizer()

%matplotlib inline

In [ ]:
# Load data
df_gbfs = pd.read_csv('../../results/chapter3/3_1_1_a_gbfs_params.csv')

# Use AdvancedKnapsackVisualizer for consistent visualization
fig = visualizer.plot_gbfs_parameter_impact(df_gbfs, save_path=None)
plt.show()

# Display summary table
print("\nGBFS Parameter Impact Summary:")
print("="*80)
print(df_gbfs[['max_states', 'value', 'time', 'value_std']].to_string(index=False))

### Nhận xét GBFS (Greedy Best-First Search):

**✅ IMPLEMENTATION ĐÚNG LÝ THUYẾT:**

Code hiện tại đã implement **đầy đủ Greedy Best-First Search** với:

1. **State Representation**: Class `KnapsackState` 
   - `selected_indices`: Items đã chọn
   - `total_weight`, `total_value`: Tổng trọng lượng/giá trị
   - `regions_covered`: Set các vùng đã cover
   - `next_item_idx`: Index item tiếp theo để expand

2. **Priority Queue (Open Set)**: 
   - Sử dụng `heapq` (min-heap)
   - Sort theo negative fitness (để lấy state có fitness cao nhất)
   - Format: `(-fitness, state_id, state)`

3. **Closed Set**:
   - Track các states đã visit để tránh revisit
   - Hash dựa trên `selected_indices`

4. **State Expansion**:
   - Từ mỗi state, thử add từng item còn lại
   - Tạo new states và push vào open set
   - Cho phép 20% overflow để exploration

5. **Multi-Objective Fitness** (giống BPSO):
   ```python
   fitness = alpha * f1_norm + beta * f2_norm - penalty
   # alpha=0.7, beta=0.3
   # f1 = Total Revenue (normalized)
   # f2 = Region Coverage (normalized)
   # penalty = 10.0 * overflow_ratio nếu vượt capacity
   ```

**Quan sát từ data:**
- **Max States có tác dụng rõ ràng**: Khi tăng max_states, quality tăng
  - 100 states: Value ≈ 52K, Fitness ≈ 0.44
  - 5000 states: Value ≈ 84K, Fitness ≈ 0.52 (+60% improvement!)
- Thời gian thực thi **tăng tuyến tính** với states_explored
- GBFS explore state tree một cách systematic theo fitness

**So sánh với BPSO:**
- GBFS: **Deterministic exploration** theo fitness gradient
- BPSO: **Stochastic optimization** với swarm intelligence
- GBFS: Nhanh hơn, ổn định hơn, nhưng có thể stuck ở local optima
- BPSO: Chậm hơn, ngẫu nhiên hơn, nhưng có khả năng escape local optima

**Kết luận:**
- GBFS hiện tại **đúng chuẩn lý thuyết** (priority queue + state expansion)
- Max States parameter **quan trọng** - giới hạn computational cost
- Trade-off: Quality vs Time - tăng max_states để quality tốt hơn
- **Khuyến nghị**: max_states = 2500-5000 cho balance tốt

### Nhận xét tóm tắt BPSO Swarm Size:

**Quan sát:**
- Chất lượng giải pháp **dao động mạnh** khi tăng Swarm Size
- **Vấn đề stability nghiêm trọng**: Std Dev = 4,888 - 10,519 (6-13% của value)
- Thời gian thực thi **tăng tuyến tính** với Swarm Size (10x)
- Swarm Size = 30 cho **value cao nhất** (90,577) và **cân bằng tốt nhất**

**Kết luận:**
- Swarm nhỏ (10-20): **Nhanh** nhưng **cực kỳ không ổn định**, quality thấp
- **Swarm = 30: Lựa chọn tốt nhất** (quality cao, time OK, stability chấp nhận được)
- Swarm lớn (100+): **Chất lượng tương đương** nhưng **chậm hơn nhiều** - không đáng
- **Khuyến nghị**: Swarm Size = 30-50, chạy nhiều lần để đảm bảo kết quả tin cậy

### Nhận xét tóm tắt BPSO Iterations:

**Quan sát:**
- Chất lượng giải pháp **không tăng tuyến tính**: có điểm bất thường tại 50 (thấp nhất) và 150 (giảm)
- **Vấn đề stability**: Std Dev = 4,076 - 13,101 (5-16% của value)
- Thời gian thực thi **tăng tuyến tính** với Iterations (5x)
- Iterations = 100 cho **value cao nhất** (92,798)

**Kết luận:**
- Iterations thấp (20-30): **Nhanh** nhưng **chưa hội tụ**, không ổn định
- **⚠️ TRÁNH Iterations = 50**: Premature convergence - value thấp nhất!
- **Iterations = 100: Quality cao nhất** - đáng để chờ
- Iterations = 150: **Giảm quality** (over-exploration?)
- **Khuyến nghị**: Iterations = 70-100 cho kết quả tốt

**So sánh với Swarm Size:**
- Iterations ảnh hưởng **mạnh hơn** đến chất lượng (range ±18% vs ±17%)
- Thời gian tăng **chậm hơn** (5x vs 10x)
- **Ưu tiên tối ưu Iterations trước**, sau đó điều chỉnh Swarm Size

### 🎯 Khuyến nghị cấu hình tham số (Quick Reference):

| Thuật toán | Tham số | Khuyến nghị | Lý do |
|------------|---------|-------------|-------|
| **GBFS** | Max States | **2500-5000** | Balance giữa quality và time |
| **BPSO** | Swarm Size | **30** | Value cao nhất (90,577), cân bằng tốt |
| **BPSO** | Iterations | **100** | Quality cao nhất (92,798), tránh premature convergence |
| **BPSO** | Inertia (w) | **0.7** (default) | Cân bằng exploration/exploitation |

### 📈 Trade-offs quan trọng:

**GBFS (Greedy Best-First Search):**
- ✅ Deterministic, ổn định, có thể reproduce
- ✅ Nhanh hơn BPSO (~10-100x tùy max_states)
- ✅ Dùng cùng fitness function với BPSO (alpha=0.7, beta=0.3)
- ✅ True search algorithm với priority queue
- ⚠️ Quality phụ thuộc max_states (trade-off time vs quality)
- ⚠️ Có thể stuck ở local optima (greedy nature)

**BPSO (Metaheuristic):**
- ✅ Không cần heuristic, linh hoạt với bài toán phức tạp
- ✅ Stochastic search có khả năng escape local optima
- ⚠️ Chậm hơn, không ổn định (std 5-16%), cần chạy nhiều lần

### 🔬 Kết luận chính:

1. **GBFS giờ đây là TRUE GBFS** với priority queue, state tree, closed set
2. **Cùng fitness function với BPSO** (alpha=0.7, beta=0.3) - đồng nhất!
3. **Max States quan trọng**: 100→5000 tăng value từ 52K→84K (+60%)
4. **BPSO cần config cẩn thận**: Iterations=100, Swarm=30, chạy nhiều lần
5. **Dùng GBFS** khi cần deterministic, nhanh, với max_states đủ lớn
6. **Dùng BPSO** khi cần escape local optima, bài toán phức tạp

In [ ]:
# Tải dữ liệu kích thước bầy đàn BPSO
df_swarm = pd.read_csv('../../results/chapter3/3_1_1_b_bpso_swarm_size.csv')
# Đổi tên cột để rõ ràng hơn
df_swarm = df_swarm.rename(columns={'param_value': 'swarm_size'})

print("Phân tích kích thước bầy đàn BPSO")
print("="*60)
print(df_swarm[['swarm_size', 'value', 'time', 'value_std']].to_string(index=False))
print("\n")

---

## 📊 TỔNG KẾT & KHUYẾN NGHỊ

### Tổng kết phân tích tham số

In [ ]:
# Load BPSO swarm size data
df_swarm = pd.read_csv('../../results/chapter3/3_1_1_b_bpso_swarm_size.csv')

# Use AdvancedKnapsackVisualizer
fig = visualizer.plot_bpso_parameter_impact(df_swarm, param_name='n_particles', save_path=None)
plt.show()

# Display summary
print("\nBPSO Swarm Size Impact Summary:")
print("="*80)
print(df_swarm[['param_value', 'value', 'time', 'value_std']].to_string(index=False))

In [ ]:
# Tải dữ liệu số vòng lặp BPSO
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')
print("Phân tích số vòng lặp BPSO")
print("="*60)
print(df_iter.to_string(index=False))
print("\n")

# Load BPSO Iterations data
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')
# Rename column for clarity
df_iter = df_iter.rename(columns={'param_value': 'iterations'})

print("BPSO Iterations Analysis")
print("="*60)
print(df_iter[['iterations', 'value', 'time', 'value_std']].to_string(index=False))
print("\n")

In [ ]:
# Load BPSO iterations data
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')

# Use AdvancedKnapsackVisualizer
fig = visualizer.plot_bpso_parameter_impact(df_iter, param_name='max_iterations', save_path=None)
plt.show()

# Display summary
print("\nBPSO Iterations Impact Summary:")
print("="*80)
print(df_iter[['param_value', 'value', 'time', 'value_std']].to_string(index=False))

In [ ]:
### Nhận xét BPSO Iterations:

**Quan sát từ data thực tế:**
- Chất lượng giải pháp **không tăng tuyến tính** khi tăng Iterations:
  - 20 iterations: 81,328
  - 30 iterations: 83,455
  - **50 iterations: 78,787 (THẤP NHẤT!)** ⚠️
  - 70 iterations: 88,496
  - **100 iterations: 92,798 (CAO NHẤT!)** ✅
  - 150 iterations: 87,824 (giảm 5.4%)
- **⚠️ Vấn đề stability:**
  - Độ lệch chuẩn rất cao: 4,076 - 13,101 (5-16% so với value!)
  - Iterations = 20: std = 13,101 (16.1%) - **CỰC KỲ KHÔNG ỔN ĐỊNH**
  - Iterations = 150: std = 4,458 (5.1%) - tốt nhất nhưng value thấp
- Thời gian thực thi **tăng tuyến tính**: 0.009s → 0.046s (tăng 5x)

**Phân tích chi tiết:**
| Iterations | Value | Std Dev | Stability | Time | Nhận xét |
|------------|-------|---------|-----------|------|----------|
| 20 | 81,328 | 13,101 | ❌ Rất kém | 0.009s | Chưa hội tụ |
| 30 | 83,455 | 9,372 | ❌ Kém | 0.010s | Vẫn chưa đủ |
| 50 | 78,787 | 4,076 | ✅ Ổn định | 0.016s | **⚠️ BẪY NGẦM!** |
| 70 | 88,496 | 12,677 | ❌ Kém | 0.022s | Không ổn định |
| **100** | **92,798** | **9,097** | **⚠️ OK** | **0.031s** | **⭐ Best quality** |
| 150 | 87,824 | 4,458 | ✅ Ổn định | 0.046s | Overfitting? |

**⚠️ Hiện tượng bất thường tại Iterations = 50:**
- Value **thấp nhất** (78,787) trong khi stability lại **tốt nhất** (std = 4,076)
- **Nguyên nhân có thể:**
  - 50 iterations chưa đủ để explore solution space → **hội tụ sớm vào local optima**
  - Std thấp vì particles bị "mắc kẹt" ở cùng một vùng (premature convergence)
  - Đây là **false stability** - ổn định nhưng ở giải pháp kém

**Hiện tượng giảm chất lượng tại 150 iterations:**
- Value giảm từ 92,798 (100 iters) → 87,824 (150 iters) = **-5.4%**
- **Nguyên nhân:**
  - Có thể do over-exploration → particles "phá vỡ" giải pháp tốt đã tìm được
  - BPSO có exploration/exploitation trade-off - 150 iters có thể quá nhiều cho bài toán này
  - Random nature của BPSO - cần chạy nhiều lần để kết luận

**Kết luận:**
- **Iterations = 100** cho **chất lượng tốt nhất** (92,798)
- Iterations = 70 là **lựa chọn an toàn** nếu cần cân bằng (88,496, 0.022s)
- **TRÁNH Iterations = 50** - mặc dù stable nhưng quality kém (premature convergence)
- Iterations > 100: không cải thiện, thậm chí **giảm quality**

**So sánh với Swarm Size:**
- **Iterations ảnh hưởng mạnh hơn** đến chất lượng: range 78K-93K (±18%) vs 78K-91K (±17%)
- Thời gian tăng **chậm hơn**: 5x vs 10x (Swarm Size)
- **Ưu tiên tối ưu Iterations trước**, sau đó điều chỉnh Swarm Size
- **Khuyến nghị**: Iterations = 70-100, Swarm Size = 30-50

## 3.1.1.d: BPSO - Inertia Weight (w) Impact

Inertia weight (w) kiểm soát sự cân bằng giữa exploration và exploitation trong PSO:
- **w cao (0.9)**: Exploration mạnh, global search, có thể escape local optima
- **w thấp (0.3)**: Exploitation mạnh, local search, hội tụ nhanh
- **w trung bình (0.5-0.7)**: Cân bằng giữa exploration và exploitation

**Test setup:**
- w_values = [0.3, 0.5, 0.7, 0.9]
- n_particles = 30, max_iterations = 50
- 5 runs per w value

In [ ]:
# Load BPSO Inertia Weight data
df_w = pd.read_csv('../../results/chapter3/3_1_1_d_bpso_w.csv')

# Use AdvancedKnapsackVisualizer
fig = visualizer.plot_bpso_parameter_impact(df_w, param_name='w', save_path=None)
plt.show()

# Display table
print("\n📊 BPSO Inertia Weight (w) Impact:")
print(df_w.to_string(index=False))

In [ ]:
# Analysis of w impact
print("\n🔍 PHÂN TÍCH INERTIA WEIGHT (w):\n")

# Value analysis
best_w = df_w.loc[df_w['value'].idxmax()]
worst_w = df_w.loc[df_w['value'].idxmin()]

print(f"✅ Best w = {best_w['param_value']} → Value = {best_w['value']:.2f} (±{best_w['value_std']:.2f})")
print(f"❌ Worst w = {worst_w['param_value']} → Value = {worst_w['value']:.2f} (±{worst_w['value_std']:.2f})")
print(f"📈 Improvement: {((best_w['value'] - worst_w['value']) / worst_w['value'] * 100):.1f}%")

# Time analysis
print(f"\n⏱️  Time:")
print(f"   Fastest: w={df_w.loc[df_w['time'].idxmin(), 'param_value']} → {df_w['time'].min():.4f}s")
print(f"   Slowest: w={df_w.loc[df_w['time'].idxmax(), 'param_value']} → {df_w['time'].max():.4f}s")

# Stability analysis
print(f"\n📊 Stability (Std Dev):")
for idx, row in df_w.iterrows():
    cv = (row['value_std'] / row['value']) * 100
    print(f"   w={row['param_value']}: ±{row['value_std']:.2f} (CV={cv:.1f}%)")

### Nhận xét BPSO Inertia Weight (w):

**Khám phá chính:**

1. **w = 0.9 (High Exploration)**
   - Quality cao nhất (~92K)
   - Exploration mạnh → tìm được global optima tốt hơn
   - Phù hợp với bài toán có nhiều local optima

2. **w = 0.7 (Balanced)**
   - Quality tốt (~70K)
   - Cân bằng exploration và exploitation
   - Ổn định hơn w=0.9

3. **w = 0.5 (Medium Exploitation)**
   - Quality trung bình (~67K)
   - Exploitation nhiều hơn, hội tụ nhanh
   - Có thể bị stuck ở local optima

4. **w = 0.3 (Low Exploration)**
   - Quality thấp nhất (~66K)
   - Exploitation quá mạnh → premature convergence
   - Không phù hợp với bài toán phức tạp

**Trade-offs:**
- ⬆️ w cao → exploration mạnh → quality tốt hơn, nhưng có thể chậm hội tụ
- ⬇️ w thấp → exploitation mạnh → hội tụ nhanh, nhưng dễ bị stuck
- 🎯 w=0.7-0.9 tối ưu cho bài toán này

## 📊 So sánh tổng hợp 4 experiments

In [ ]:
# Load all data files
df_gbfs = pd.read_csv('../../results/chapter3/3_1_1_a_gbfs_params.csv')
df_swarm = pd.read_csv('../../results/chapter3/3_1_1_b_bpso_swarm_size.csv')
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')
df_w = pd.read_csv('../../results/chapter3/3_1_1_d_bpso_w.csv')

# Comparative visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('3.1.1: Parameter Impact Comparison - GBFS vs BPSO', fontsize=16, fontweight='bold')

# Plot 1: GBFS max_states
ax1 = axes[0, 0]
ax1.plot(df_gbfs['max_states'], df_gbfs['value'], marker='o', linewidth=2, markersize=8, color='#e74c3c', label='Value')
ax1.set_xlabel('Max States', fontweight='bold')
ax1.set_ylabel('Value', fontweight='bold', color='#e74c3c')
ax1.tick_params(axis='y', labelcolor='#e74c3c')
ax1.set_title('GBFS: Max States Impact', fontweight='bold', pad=10)
ax1.grid(True, alpha=0.3)

ax1_time = ax1.twinx()
ax1_time.plot(df_gbfs['max_states'], df_gbfs['time'], marker='s', linewidth=2, markersize=6, color='#3498db', linestyle='--', label='Time')
ax1_time.set_ylabel('Time (s)', fontweight='bold', color='#3498db')
ax1_time.tick_params(axis='y', labelcolor='#3498db')

# Plot 2: BPSO swarm size
ax2 = axes[0, 1]
ax2.errorbar(df_swarm['param_value'], df_swarm['value'], yerr=df_swarm['value_std'], 
             marker='o', linewidth=2, markersize=8, color='#9b59b6', capsize=5, label='Value ± Std')
ax2.set_xlabel('Swarm Size (n_particles)', fontweight='bold')
ax2.set_ylabel('Value', fontweight='bold', color='#9b59b6')
ax2.tick_params(axis='y', labelcolor='#9b59b6')
ax2.set_title('BPSO: Swarm Size Impact', fontweight='bold', pad=10)
ax2.grid(True, alpha=0.3)

ax2_time = ax2.twinx()
ax2_time.plot(df_swarm['param_value'], df_swarm['time'], marker='s', linewidth=2, markersize=6, color='#3498db', linestyle='--', label='Time')
ax2_time.set_ylabel('Time (s)', fontweight='bold', color='#3498db')
ax2_time.tick_params(axis='y', labelcolor='#3498db')

# Plot 3: BPSO iterations
ax3 = axes[1, 0]
ax3.errorbar(df_iter['param_value'], df_iter['value'], yerr=df_iter['value_std'],
             marker='o', linewidth=2, markersize=8, color='#16a085', capsize=5, label='Value ± Std')
ax3.set_xlabel('Max Iterations', fontweight='bold')
ax3.set_ylabel('Value', fontweight='bold', color='#16a085')
ax3.tick_params(axis='y', labelcolor='#16a085')
ax3.set_title('BPSO: Iterations Impact', fontweight='bold', pad=10)
ax3.grid(True, alpha=0.3)

ax3_time = ax3.twinx()
ax3_time.plot(df_iter['param_value'], df_iter['time'], marker='s', linewidth=2, markersize=6, color='#3498db', linestyle='--', label='Time')
ax3_time.set_ylabel('Time (s)', fontweight='bold', color='#3498db')
ax3_time.tick_params(axis='y', labelcolor='#3498db')

# Plot 4: BPSO inertia weight
ax4 = axes[1, 1]
ax4.errorbar(df_w['param_value'], df_w['value'], yerr=df_w['value_std'],
             marker='o', linewidth=2, markersize=8, color='#e67e22', capsize=5, label='Value ± Std')
ax4.set_xlabel('Inertia Weight (w)', fontweight='bold')
ax4.set_ylabel('Value', fontweight='bold', color='#e67e22')
ax4.tick_params(axis='y', labelcolor='#e67e22')
ax4.set_title('BPSO: Inertia Weight Impact', fontweight='bold', pad=10)
ax4.grid(True, alpha=0.3)

ax4_time = ax4.twinx()
ax4_time.plot(df_w['param_value'], df_w['time'], marker='s', linewidth=2, markersize=6, color='#3498db', linestyle='--', label='Time')
ax4_time.set_ylabel('Time (s)', fontweight='bold', color='#3498db')
ax4_time.tick_params(axis='y', labelcolor='#3498db')

plt.tight_layout()
plt.show()

# Comparative insights
print("\n📊 COMPARATIVE INSIGHTS:")
print("-" * 100)
print(f"   GBFS Max States Impact:    Value change = {((df_gbfs['value'].max() - df_gbfs['value'].min()) / df_gbfs['value'].min() * 100):.1f}%, Time improvement = {((df_gbfs.iloc[0]['time'] - df_gbfs.iloc[-1]['time']) / df_gbfs.iloc[0]['time'] * 100):.1f}%")
print(f"   BPSO Swarm Size Impact:    Value change = {((df_swarm['value'].max() - df_swarm['value'].min()) / df_swarm['value'].min() * 100):.1f}%, Time increase = {((df_swarm['time'].max() - df_swarm['time'].min()) / df_swarm['time'].min() * 100):.0f}%")
print(f"   BPSO Iterations Impact:    Value change = {((df_iter['value'].max() - df_iter['value'].min()) / df_iter['value'].min() * 100):.1f}%, Time increase = {((df_iter['time'].max() - df_iter['time'].min()) / df_iter['time'].min() * 100):.0f}%")
print(f"   BPSO Inertia Weight Impact: Value change = {((df_w['value'].max() - df_w['value'].min()) / df_w['value'].min() * 100):.1f}%, Time change = {((df_w['time'].max() - df_w['time'].min()) / df_w['time'].min() * 100):.1f}%")
print("-" * 100)

### 🎯 Khuyến nghị cấu hình tham số (dựa trên data thực tế):

| Thuật toán | Tham số | Giá trị khuyến nghị | Lý do | Value đạt được |
|------------|---------|---------------------|-------|----------------|
| **GBFS** | Max States | **2500-5000** | Balance quality và time tốt nhất | 74K-84K |
| **BPSO** | Swarm Size | **30** | Cân bằng tốt nhất (quality cao, time OK) | 90,577 |
| **BPSO** | Swarm Size | **100** (nếu cần quality cao) | Quality cao nhất, chấp nhận chậm 3.4x | 91,222 |
| **BPSO** | Iterations | **100** | Quality cao nhất, đáng chờ | 92,798 |
| **BPSO** | Iterations | **70** (nếu cần nhanh) | Cân bằng tốt, tránh premature convergence | 88,496 |
| **BPSO** | Inertia Weight (w) | **0.9** | Exploration mạnh, tìm global optima tốt nhất | 92,012 |
| **BPSO** | Inertia Weight (w) | **0.7** (balanced) | Cân bằng exploration-exploitation | 69,613 |

**⚠️ Lưu ý:**
- **GBFS**: Max States càng cao → quality càng tốt (nhưng chậm hơn)
- **BPSO**: Độ lệch chuẩn cao (5-16%) → CẦN CHẠY NHIỀU LẦN và lấy trung bình
- **BPSO w parameter**: w=0.9 cho exploration mạnh, w=0.3 cho exploitation nhanh
- **Cả hai** đều dùng fitness function: `alpha=0.7, beta=0.3`

### 📈 Trade-offs quan trọng:

**GBFS (TRUE Greedy Best-First Search):**
- ✅ **Ưu điểm**: 
  - **TRUE search algorithm** với priority queue, state tree, closed set
  - **Dùng cùng fitness function với BPSO** (alpha=0.7, beta=0.3) - đồng nhất!
  - **Deterministic** - kết quả có thể reproduce với cùng max_states
  - Nhanh hơn BPSO (~10-100x tùy max_states)
  - Max States parameter có ý nghĩa: giới hạn search space
  - Quality tăng khi tăng max_states (100→5000: 52K→84K, +60%)
- ⚠️ **Nhược điểm**: 
  - Greedy nature → có thể stuck ở local optima
  - Quality phụ thuộc max_states (trade-off time vs quality)
  - Exploration theo fitness gradient (không random như PSO)

**BPSO (Metaheuristic Optimization):**
- ✅ **Ưu điểm**: 
  - **Stochastic search** - có khả năng escape local optima
  - Không cần heuristic - tự khám phá solution space
  - Có thể xử lý bài toán phức tạp với nhiều ràng buộc
  - Linh hoạt, có thể tìm giải pháp "không trực quan"
  - **4 tham số quan trọng**: Swarm Size, Iterations, w, c1/c2
- ⚠️ **Nhược điểm**: 
  - **Chậm hơn GBFS** (nhưng ít hơn so với old Greedy)
  - **CỰC KỲ KHÔNG ỔN ĐỊNH** (std: 4K-13K)
  - Cần tuning cẩn thận 4 tham số
  - **Stochastic** - kết quả khác nhau mỗi lần chạy

### 🔬 Kết luận chính:

1. **✅ GBFS giờ đúng chuẩn lý thuyết:**
   - Priority queue (heapq) cho open set
   - Closed set để avoid revisiting
   - State expansion với nhiều child states
   - Max States parameter QUAN TRỌNG và có tác dụng

2. **✅ GBFS và BPSO dùng CÙNG fitness function:**
   - `fitness = 0.7 * revenue_norm + 0.3 * region_coverage_norm - penalty`
   - Đồng nhất về objective function
   - So sánh công bằng giữa 2 thuật toán

3. **4 tham số quan trọng của BPSO:**
   - **Swarm Size**: 30-100 (trade-off quality vs time)
   - **Iterations**: 50-150 (trade-off convergence vs time)
   - **Inertia Weight (w)**: 0.7-0.9 (exploration vs exploitation)
   - **c1/c2**: Cognitive và social coefficients (chưa test)

4. **Khi nào dùng thuật toán nào?**
   
   **Dùng GBFS khi:**
   - Cần tốc độ và tính deterministic
   - Muốn control computational cost chính xác (max_states)
   - Bài toán có fitness landscape tương đối smooth
   - Cần kết quả ổn định, có thể reproduce
   
   **Dùng BPSO khi:**
   - Bài toán phức tạp, nhiều local optima
   - Có nhiều ràng buộc phức tạp
   - Chấp nhận trade-off: chậm hơn + ngẫu nhiên hơn
   - Cần explore solution space toàn diện
   - Sẵn sàng chạy nhiều lần để lấy kết quả tin cậy
   - Có thể tune 4 tham số (Swarm, Iterations, w, c1/c2)

5. **So sánh quality trên test case này:**
   - GBFS (max_states=5000): ~84K, deterministic
   - BPSO (best config w=0.9, iter=100): ~93K, nhưng std cao (±9K)
   - Cả hai đều sử dụng cùng multi-objective function
   - BPSO có potential cao hơn nhưng không ổn định
   - **w=0.9** là best parameter cho exploration mạnh